In [1]:
!pip install pyDOE
!pip install cma

  Created wheel for pyDOE: filename=pyDOE-0.3.8-cp36-none-any.whl size=18178 sha256=618e3818424efed9dc57f9dd3d56f878f24b08f5d6a946c8c78f888bc5c0d357
  Stored in directory: /root/.cache/pip/wheels/7c/c8/58/a6493bd415e8ba5735082b5e0c096d7c1f2933077a8ce34544
Successfully built pyDOE
     |████████████████████████████████| 245kB 2.8MB/s 


In [0]:
import pyDOE
from scipy import stats
import pandas as pd
import numpy as np
from scipy.optimize import minimize
from scipy.optimize import Bounds
from collections import namedtuple
import cma
from sklearn.preprocessing import MinMaxScaler
import scipy.stats.distributions as dist
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C
from sklearn.gaussian_process.kernels import DotProduct
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.linear_model import ElasticNet
import bbobbenchmarks as bn
import sklearn
from sklearn.preprocessing import PolynomialFeatures

## Surrogate Models & Helper Functions

In [0]:
ValueRange = namedtuple('ValueRange', ['min', 'max'])

def determinerange(values):
    """Determine the range of values in each dimension"""
    return ValueRange(np.min(values, axis=0), np.max(values, axis=0))


def linearscaletransform(values, *, range_in=None, range_out=ValueRange(0, 1), scale_only=False):
    """Perform a scale transformation of `values`: [range_in] --> [range_out]"""

    if range_in is None:
        range_in = determinerange(values)
    elif not isinstance(range_in, ValueRange):
        range_in = ValueRange(*range_in)

    if not isinstance(range_out, ValueRange):
        range_out = ValueRange(*range_out)

    scale_out = range_out.max - range_out.min
    scale_in = range_in.max - range_in.min

    if scale_only:
        scaled_values = (values / scale_in) * scale_out
    else:
        scaled_values = (values - range_in.min) / scale_in
        scaled_values = (scaled_values * scale_out) + range_out.min

    return scaled_values


''' F3 '''
def F3(X):
    f = bn.F3()
    X = np.array(X)
    return f(X)

''' Latin HyperCube Sampling Design of Experiment '''
def DOE(n_obs, dim):
    np.random.seed(0)
    lhd = pyDOE.lhs(n=dim, samples=n_obs, criterion='m')
    X = [lhd[:,idx] for idx in range(dim)]
    return X

def create_basis_function(data):
    true = np.array(data['Y'])
    data = pd.DataFrame(np.atleast_2d(PolynomialFeatures(degree=2).fit_transform(data.iloc[:,:-1])))
    data['Y'] = pd.Series(true)
    return data


''' Create Basis Functions '''
def create_function_basis(x):
    return np.atleast_2d(PolynomialFeatures(degree=2).fit_transform(x.reshape(1,-1)))


''' Elastic Net Regression '''
def elastic_net(train_data,test_data):
    scaler =  MinMaxScaler().fit(np.r_[train_data.iloc[:,:-1].values])
    regr = ElasticNet(alpha= 0.9040895336552447 ,random_state=0 , l1_ratio=0.98989898989899, fit_intercept =True, max_iter=3000,selection='random').fit(scaler.transform ( np.array(train_data.iloc[:,:-1])) ,  np.array(train_data.iloc[:,-1]))
    pred = regr.predict(scaler.transform(test_data))
    def predict(scaler, regr):
        def __predict__(x):
            x = create_function_basis(x)
            return regr.predict(scaler.transform(x))
        return __predict__
    return regr,pred, predict(scaler, regr)
    

''' Kriging'''
def kriging(train_data,test_data):
    kernel =  RBF()
    scaler = MinMaxScaler().fit(np.r_[train_data.iloc[:,:-1].values])
    gpr = GaussianProcessRegressor(kernel=kernel,n_restarts_optimizer= 15,random_state=0,
                                   normalize_y=True ).fit(scaler.transform(train_data.iloc[:,:-1]), train_data.iloc[:,-1])
    pred = gpr.predict(scaler.transform(test_data))
    def predict(scaler, gpr):
        def __predict__(x):
            x = np.atleast_2d(x)
            return gpr.predict(scaler.transform(x))
        return __predict__
    return gpr,pred, predict(scaler,gpr)


''' Support Vector Regression'''
def _SVR(train_data,test_data):
    scaler = MinMaxScaler().fit(np.r_[train_data.iloc[:,:-1].values])
    gpr = sklearn.svm.SVR(kernel='rbf', gamma = 37.213462 , C = 1000.000000 ,max_iter=1500).fit( scaler.transform(train_data.iloc[:,:-1]), train_data.iloc[:,-1])
    pred = gpr.predict(scaler.transform(test_data))
    def predict(scaler, gpr):
        def __predict__(x):
            x = np.atleast_2d(x)
            return gpr.predict(scaler.transform(x))
        return __predict__
    return gpr,pred, predict(scaler,gpr)

## Load Training and Test Data Set initially Generated

In [0]:
path = "train_3_1000Samples.csv"
train = pd.read_csv(path).iloc[:,1:]
test = pd.read_csv('test_3_200Samples.csv').iloc[:,1:]
true = np.array(test['Y'])

## Surrogate Models

In [0]:
model_kri , pred_kri , predict_kri = kriging(train,test.iloc[:,:-1])
model_svr , pred_svr , predict_svr = _SVR(train,test.iloc[:,:-1])
train = create_basis_function(train)
test = create_basis_function(test)
model_eln , pred_eln , predict_eln = elastic_net(train,test.iloc[:,:-1])

## CMA-ES

In [6]:
Columns = ['Kri' , 'SVR' , 'ELN' ]
Cols = []
for j in range(len(Columns)):
    for i in range(1,51):
        Cols.append(Columns[j]+'_X'+str(i))
const = [ [-5] * 50, [5] * 50 ]

opt = cma.CMAOptions()
opt.set("bounds", const)
opt.set ("seed" , 0)
opt.set ("maxfevals" , 50000)

n_obs , dim =  30, 50
G = DOE(n_obs, dim)
G  = [ linearscaletransform(G[idx] , range_out=(-5,5)) for idx in range(dim) ]
G = [ G[idx].reshape(n_obs,1) for idx in range(len(G)) ]
X_Values = np.zeros([30,150])
for i in range(G[1].shape[0]):
        min_kri = cma.fmin(predict_kri , np.concatenate(G, 1)[i] , 2.5 , options=opt) [0]
        min_svr = cma.fmin(predict_svr , np.concatenate(G, 1)[i] , 2.5 , options=opt) [0]
        min_eln = cma.fmin(predict_eln , np.concatenate(G, 1)[i] , 2.5 , options=opt) [0]
        X_Values [i,:] = list(min_kri)+list(min_svr)+list(min_eln)

(7_w,15)-aCMA-ES (mu_w=4.5,w_1=34%) in dimension 50 (seed=873862, Mon Apr 13 18:39:37 2020)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     15 5.964530546041883e+03 1.0e+00 2.36e+00  2e+00  2e+00 0:00.0
    2     30 4.462280605352483e+03 1.0e+00 2.30e+00  2e+00  2e+00 0:00.0
    3     45 3.459098169132377e+03 1.1e+00 2.26e+00  2e+00  2e+00 0:00.1


/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [7983.27047345] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [6460.13215379] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')


  100   1500 -8.573336417095150e+03 1.5e+00 5.77e-01  5e-01  6e-01 0:01.7
  200   3000 -9.797086092254198e+03 1.6e+00 4.38e-02  3e-02  4e-02 0:03.4
  300   4500 -9.803077172712630e+03 1.7e+00 2.32e-03  2e-03  2e-03 0:05.1
  400   6000 -9.803095357474482e+03 1.7e+00 1.17e-04  8e-05  1e-04 0:06.8
  500   7500 -9.803095398697798e+03 1.9e+00 5.74e-06  3e-06  5e-06 0:08.5
  600   9000 -9.803095398800881e+03 2.0e+00 4.31e-07  2e-07  4e-07 0:10.2
  700  10500 -9.803095398801390e+03 2.1e+00 3.57e-08  2e-08  3e-08 0:11.9
  800  12000 -9.803095398801397e+03 2.4e+00 1.72e-08  8e-09  1e-08 0:13.6
  900  13500 -9.803095398801397e+03 2.7e+00 1.53e-08  7e-09  1e-08 0:15.3
  918  13770 -9.803095398801393e+03 2.7e+00 1.48e-08  6e-09  1e-08 0:15.6
termination on tolfun=1e-11 (Mon Apr 13 18:39:55 2020)
final/bestever f-value = -9.803095e+03 -9.803095e+03
incumbent solution: [-0.28161629 -0.09263547 -0.02315725  0.06336384 -0.86248519  0.92546084
 -0.68441681 -0.24269959 ...]
std deviations: [1.12841335e-

/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [5969.35151166] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: flat fitness (f=5969.351512, sigma=2.37e+00).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately. (iteration=1)
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: flat fitness (f=5969.351512, sigma=2.37e+00).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately. (iteration=1)
  ')')

  100   1500 -1.806728762097967e+03 1.6e+00 5.80e-01  5e-01  6e-01 0:02.0
  200   3000 -2.544161415464256e+03 2.0e+00 2.03e-01  2e-01  2e-01 0:04.1
  300   4500 -2.701031679624402e+03 2.3e+00 4.03e-02  3e-02  5e-02 0:06.1
  400   6000 -2.705107145130798e+03 2.6e+00 1.32e-02  9e-03  2e-02 0:08.2
  500   7500 -2.705451906888779e+03 3.2e+00 2.51e-03  2e-03  3e-03 0:10.2
  600   9000 -2.705471508667483e+03 3.9e+00 7.34e-04  4e-04  1e-03 0:12.3
  700  10500 -2.705472281657079e+03 4.4e+00 1.17e-04  6e-05  2e-04 0:14.3
  800  12000 -2.705472299788458e+03 4.8e+00 2.20e-05  1e-05  4e-05 0:16.3
  900  13500 -2.705472300773925e+03 5.2e+00 6.98e-06  3e-06  1e-05 0:18.4
 1000  15000 -2.705472300856796e+03 5.6e+00 1.69e-06  7e-07  3e-06 0:20.4
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
 1100  16500 -2.705472300860459e+03 6.0e+00 3.33e-07  1e-07  7e-07 0:22.4
 1200  18000 -2.705472300860633e+03 6.6e+00 6.60e-08  3e-08  1e-07 0:24.5
 1287  19305 -2.705472300860636e+03 7.0e

...

  100   1500 -2.325661110361390e+03 1.6e+00 6.19e-01  5e-01  7e-01 0:02.0
  200   3000 -2.896619712860017e+03 1.7e+00 1.07e-01  8e-02  1e-01 0:04.1
  300   4500 -2.957876622542788e+03 3.2e+00 6.77e-02  5e-02  1e-01 0:06.1
  400   6000 -2.964086599360402e+03 3.4e+00 1.32e-02  9e-03  2e-02 0:08.2
  500   7500 -2.964440091668536e+03 3.7e+00 2.75e-03  2e-03  4e-03 0:10.2
  600   9000 -2.964453799711294e+03 4.0e+00 5.47e-04  3e-04  8e-04 0:12.3
  700  10500 -2.964454469833390e+03 4.2e+00 2.26e-04  1e-04  3e-04 0:14.3
  800  12000 -2.964454576295005e+03 4.8e+00 8.79e-05  4e-05  1e-04 0:16.3
  900  13500 -2.964454585353611e+03 5.6e+00 1.34e-05  6e-06  2e-05 0:18.4
 1000  15000 -2.964454585573324e+03 6.1e+00 1.63e-06  7e-07  3e-06 0:20.4
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
 1100  16500 -2.964454585575461e+03 6.5e+00 2.05e-07  8e-08  4e-07 0:22.4
 1200  18000 -2.964454585575495e+03 6.8e+00 3.13e-08  1e-08  5e-08 0:24.5
 1237  18555 -2.964454585575496e+03 6.9e

/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [3960.5267102] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [4459.53352413] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')


  100   1500 -9.444134864070125e+03 1.4e+00 3.35e-01  3e-01  3e-01 0:02.0
  200   3000 -9.800882407769426e+03 1.6e+00 2.61e-02  2e-02  3e-02 0:04.0
  300   4500 -9.803088368188866e+03 1.7e+00 1.47e-03  1e-03  1e-03 0:06.0
  400   6000 -9.803095372751377e+03 1.8e+00 8.77e-05  5e-05  8e-05 0:08.0
  500   7500 -9.803095398586527e+03 2.0e+00 7.63e-06  4e-06  7e-06 0:10.1
  600   9000 -9.803095398800757e+03 2.0e+00 5.48e-07  3e-07  5e-07 0:12.1
  700  10500 -9.803095398801397e+03 2.1e+00 4.69e-08  2e-08  4e-08 0:14.1
  800  12000 -9.803095398801393e+03 2.4e+00 1.99e-08  1e-08  2e-08 0:16.1
  900  13500 -9.803095398801390e+03 2.7e+00 7.31e-09  3e-09  7e-09 0:18.0
 1000  15000 -9.803095398801390e+03 2.9e+00 8.43e-09  4e-09  8e-09 0:20.1
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
 1100  16500 -9.803095398801393e+03 3.2e+00 1.04e-08  5e-09  9e-09 0:22.1
 1200  18000 -9.803095398801397e+03 3.5e+00 1.25e-08  5e-09  1e-08 0:24.1
 1269  19035 -9.803095398801393e+03 3.7e

/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [5969.35151166] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: flat fitness (f=5969.351512, sigma=2.35e+00).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately. (iteration=1)
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: flat fitness (f=5969.351512, sigma=2.35e+00).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately. (iteration=1)
  ')')

  100   1500 -1.776592124349299e+03 1.6e+00 8.28e-01  7e-01  9e-01 0:02.0
  200   3000 -2.861014938885132e+03 1.9e+00 1.77e-01  1e-01  2e-01 0:04.1
  300   4500 -3.095379442608896e+03 2.9e+00 8.04e-02  6e-02  1e-01 0:06.2
  400   6000 -3.113854412046228e+03 2.8e+00 1.79e-02  1e-02  2e-02 0:08.2
  500   7500 -3.114677941862068e+03 2.8e+00 4.00e-03  2e-03  5e-03 0:10.2
  600   9000 -3.114746834479389e+03 3.5e+00 1.86e-03  1e-03  3e-03 0:12.3
  700  10500 -3.114753230277771e+03 4.1e+00 3.42e-04  2e-04  7e-04 0:14.3
  800  12000 -3.114753411224337e+03 4.6e+00 7.18e-05  4e-05  2e-04 0:16.4
  900  13500 -3.114753418574368e+03 4.7e+00 8.25e-06  4e-06  2e-05 0:18.4
 1000  15000 -3.114753418657464e+03 5.0e+00 9.92e-07  5e-07  2e-06 0:20.5
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
 1100  16500 -3.114753418658327e+03 5.2e+00 8.99e-08  4e-08  2e-07 0:22.5
 1200  18000 -3.114753418658336e+03 5.5e+00 3.00e-08  1e-08  6e-08 0:24.5
 1214  18210 -3.114753418658338e+03 5.5e

In [7]:
X_Values = pd.DataFrame(X_Values)
X_Values.columns = Cols
X_Values.to_csv('Results\\F3_X_Values.csv')
Krig_Fun = np.zeros(30)
SVM_Fun = np.zeros(30)
ELN_Fun = np.zeros(30)
for i in range(X_Values.shape[0]):
    Krig_Fun [i] = F3(X_Values.iloc[i,:50])
    SVM_Fun [i] = F3(X_Values.iloc[i,50:100])
    ELN_Fun [i] = F3(X_Values.iloc[i,100:150])
print ('Kriging')
print (stats.mode(Krig_Fun) , np.std(Krig_Fun))
print ('SVM')
print (stats.mode(SVM_Fun) , np.std(SVM_Fun))
print ('ELN')
print (stats.mode(ELN_Fun) , np.std(ELN_Fun))

Kriging
ModeResult(mode=array([1207.87207714]), count=array([1])) 3.7071418374717775e-05
SVM
ModeResult(mode=array([3458.90482768]), count=array([1])) 3352.932026171831
ELN
ModeResult(mode=array([2137.69524523]), count=array([1])) 506.8800858757234
